In [1]:
import json
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from tqdm import trange

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, RobertaModel,
    BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from typing import List
from torch.utils.data import Sampler

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [3]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cuda:0].


## Tokenizer 체크

In [15]:
model_checkpoint = 'klue/bert-base'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# num_added_toks = tokenizer.add_tokens(['\\n']) # 32000번째로 삽입

In [7]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
train_dataset = dataset['train']

In [ ]:
from tqdm import tqdm

unk_count = []
for i in tqdm(range(len(train_dataset))) :
    unk_count.append(tokenizer(train_dataset['context'][i], max_length = True)['input_ids'].count(1)) # 1 == UNK

In [ ]:
print(np.median(unk_count))
print(np.mean(unk_count))
print(np.max(unk_count))
print(np.min(unk_count))

## Training


In [4]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [5]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
train_dataset = dataset['train']

In [6]:
from datasets import Dataset
dataset = pd.read_csv('/opt/ml/data/train_dataset/Aug_Encoder.csv')
dataset = Dataset.from_pandas(dataset)

In [7]:
class CustomSampler(Sampler) :
    def __init__(self, data_source, batch_size) :
        self.data_source = data_source
        self.batch_size = batch_size

    def __iter__(self) :
        n = len(self.data_source)
        index_list = []
        while True :
            out = True
            for i in range(self.batch_size) :
                tmp_data = random.randint(0, n-1)
                index_list.append(tmp_data)
            for f, s in zip(index_list, index_list[1:]) :
                if abs(s-f) <= 2 :
                    out = False
            if out == True :
                break

        while True : # 추가 삽입
            tmp_data = random.randint(0, n-1)
            if (tmp_data not in index_list) and \
                (abs(tmp_data-index_list[-i]) > 2 for i in range(1,self.batch_size+1)) \
            : 
                index_list.append(tmp_data)
            if len(index_list) == n :
                break
        return iter(index_list)

    def __len__(self) :
        return len(self.data_source)

In [8]:
# Anwer
class DenseRetrieval:
    def __init__(self,
        args,
        dataset,
        tokenizer,
        p_encoder,
        q_encoder,
        sampler
    ):
        """
        학습과 추론에 사용될 여러 셋업을 마쳐봅시다.
        """

        self.args = args
        self.dataset = dataset

        self.tokenizer = tokenizer
        self.p_encoder = p_encoder
        self.q_encoder = q_encoder
        self.sampler = sampler

    def train(self, args=None, tokenizer = None):
        if args is None:
            args = self.args
        if tokenizer is None :
            tokenizer = self.tokenizer

        for i in (range(len(self.dataset))) :
            if i == 0 :
                q_seqs = tokenizer(
                    self.dataset[i]['question'],
                    padding='max_length',
                    max_length=512,
                    truncation = True,
                    return_tensors='pt'
                )
                p_seqs = tokenizer(
                    self.dataset[i]['context'],
                    truncation = True,
                    stride = 128,
                    padding='max_length',
                    max_length=512,
                    return_overflowing_tokens=True,
                    return_offsets_mapping=True,
                    return_tensors='pt'
                )

                p_seqs.pop('overflow_to_sample_mapping')
                p_seqs.pop('offset_mapping')

                for k in q_seqs.keys() :
                    q_seqs[k] = q_seqs[k].tolist()
                    p_seqs[k] = p_seqs[k].tolist()
                
                initial_p_seqs_length = len(p_seqs['input_ids'])
                for j in range(len(p_seqs['input_ids'])) :
                    q_seqs['input_ids'].append(q_seqs['input_ids'][0])
                    q_seqs['token_type_ids'].append(q_seqs['token_type_ids'][0])
                    q_seqs['attention_mask'].append(q_seqs['attention_mask'][0])
                    p_seqs['input_ids'].append(p_seqs['input_ids'][j])
                    p_seqs['token_type_ids'].append(p_seqs['token_type_ids'][j])
                    p_seqs['attention_mask'].append(p_seqs['attention_mask'][j])

                q_seqs['input_ids'] = q_seqs['input_ids'][1:]
                q_seqs['token_type_ids'] = q_seqs['token_type_ids'][1:]
                q_seqs['attention_mask'] = q_seqs['attention_mask'][1:]
                p_seqs['input_ids'] = p_seqs['input_ids'][initial_p_seqs_length:]
                p_seqs['token_type_ids'] = p_seqs['token_type_ids'][initial_p_seqs_length:]
                p_seqs['attention_mask'] = p_seqs['attention_mask'][initial_p_seqs_length:]

            else :
                tmp_q_seq = tokenizer(
                    self.dataset[i]['question'],
                    padding='max_length',
                    max_length=512,
                    truncation = True,
                    return_tensors='pt')
                tmp_p_seq = tokenizer(
                    self.dataset[i]['context'],
                    truncation = True,
                    stride = 128,
                    padding='max_length',
                    max_length=512,
                    return_overflowing_tokens=True,
                    return_offsets_mapping=True,
                    return_tensors='pt'
                )

                tmp_p_seq.pop('overflow_to_sample_mapping')
                tmp_p_seq.pop('offset_mapping')

                for k in tmp_p_seq.keys() :
                    tmp_p_seq[k] = tmp_p_seq[k].tolist()
                    tmp_q_seq[k] = tmp_q_seq[k].tolist()


                for j in range(len(tmp_p_seq['input_ids'])) :
                    q_seqs['input_ids'].append(tmp_q_seq['input_ids'][0])
                    q_seqs['token_type_ids'].append(tmp_q_seq['token_type_ids'][0])
                    q_seqs['attention_mask'].append(tmp_q_seq['attention_mask'][0])
                    p_seqs['input_ids'].append(tmp_p_seq['input_ids'][j])
                    p_seqs['token_type_ids'].append(tmp_p_seq['token_type_ids'][j])
                    p_seqs['attention_mask'].append(tmp_p_seq['attention_mask'][j])

        for k in q_seqs.keys() :
            q_seqs[k] = torch.tensor(q_seqs[k])
            p_seqs[k] = torch.tensor(p_seqs[k])

        train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])
        sampler = self.sampler(train_dataset, args.per_device_train_batch_size)
        train_dataloader = DataLoader(train_dataset,
                                      batch_size=args.per_device_train_batch_size,
                                      sampler = sampler)

        no_decay = ["bias" ,"LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
            {"params": [p for n, p in self.q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            # eps=args.adam_epsilon
        )

        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
        
        global_step = 0

        self.p_encoder.zero_grad()
        self.q_encoder.zero_grad()
        # torch.cuda.empty_cache()

        train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
        self.q_encoder.train()
        self.p_encoder.train()
        for epoch, _ in enumerate(train_iterator):
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            loss_value=0 # Accumulation할 때 진행
            losses = 0
            for step, batch in enumerate(epoch_iterator):
                if torch.cuda.is_available():
                    batch = tuple(t.cuda() for t in batch)

                p_inputs = {'input_ids': batch[0],
                            'attention_mask': batch[1],
                            'token_type_ids': batch[2]
                            }
                
                q_inputs = {'input_ids': batch[3],
                            'attention_mask': batch[4],
                            'token_type_ids': batch[5]}

                p_outputs = self.p_encoder(**p_inputs)  # (batch_size, emb_dim)
                q_outputs = self.q_encoder(**q_inputs)  # (batch_size, emb_dim)

                # Calculate similarity score & loss
                sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)

                # target: position of positive samples = diagonal element 
                # targets = torch.arange(0, args.per_device_train_batch_size).long()
                targets = torch.arange(0, len(p_inputs['input_ids'])).long()
                
                if torch.cuda.is_available():
                    targets = targets.to('cuda')
                    
                sim_scores = F.log_softmax(sim_scores, dim=1)

                loss = F.nll_loss(sim_scores, targets)
                losses += loss.item()
                if step % 100 == 0 :
                    print(f'{epoch}epoch loss: {losses/(step+1)}') # Accumulation할 경우 주석처리

                # ################ACCUMULATION###############################
                # if (step+1) % args.gradient_accumulation_steps == 0 :
                #     optimizer.step()
                #     scheduler.step()
                #     self.q_encoder.zero_grad()
                #     self.p_encoder.zero_grad()

                # losses += loss.item()
                # if (step+1) % 64 == 0:
                #     train_loss = losses / 64
                #     print(f'training loss: {train_loss:4.4}')
                #     losses = 0
                # ###########################################################
                self.q_encoder.zero_grad()
                self.p_encoder.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()

                # global_step += 1
                
                #torch.cuda.empty_cache()
                del p_inputs, q_inputs

        return self.p_encoder, self.q_encoder

In [9]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    num_train_epochs=20,
    weight_decay=0.01
)
model_checkpoint = "klue/bert-base"

# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# num_added_toks = tokenizer.add_tokens(['\\n']) # 32000번째로 삽입
p_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)
q_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.de

In [10]:
# sampler = CustomSampler()
# Retriever는 아래와 같이 사용할 수 있도록 코드를 짜봅시다.
retriever = DenseRetrieval(
    args=args,
    dataset=dataset,
    tokenizer=tokenizer,
    p_encoder=p_encoder,
    q_encoder=q_encoder,
    sampler = CustomSampler
)
p_encoder, q_encoder = retriever.train()

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

0epoch loss: 18.882301330566406
0epoch loss: 2.747803631681248
0epoch loss: 1.8054619641900433
0epoch loss: 1.4123133636636955
0epoch loss: 1.1848713733812755
0epoch loss: 1.0417346696662655
0epoch loss: 0.9306071474814306
0epoch loss: 0.8390770195624015
0epoch loss: 0.7728449736591047
0epoch loss: 0.7189405193238176
0epoch loss: 0.6735993443199785
0epoch loss: 0.6318205580153833
0epoch loss: 0.6004730541335069
0epoch loss: 0.5747150797251602
0epoch loss: 0.5516593690286893
0epoch loss: 0.5282819543282045
0epoch loss: 0.509971809012273
0epoch loss: 0.49323402389324744
0epoch loss: 0.47776585388611437
0epoch loss: 0.4650395929750753
0epoch loss: 0.4524701598909352
0epoch loss: 0.44089864972736387
0epoch loss: 0.42957448807852655
0epoch loss: 0.419596689456713
0epoch loss: 0.41118150971156925
0epoch loss: 0.4011327781237818
0epoch loss: 0.3935750661385359


Epoch:   5%|▌         | 1/20 [49:01<15:31:19, 2941.01s/it]

1epoch loss: 0.7511616349220276
1epoch loss: 0.16321438028117513
1epoch loss: 0.14410351590017106
1epoch loss: 0.1356538531519461
1epoch loss: 0.12847854031590708
1epoch loss: 0.13271724936847915
1epoch loss: 0.12802190902596805
1epoch loss: 0.1296040934319382
1epoch loss: 0.1321448438298585
1epoch loss: 0.1359717246027125
1epoch loss: 0.13792185790834746
1epoch loss: 0.14020746292643907
1epoch loss: 0.13952978648308753
1epoch loss: 0.1402367392770272
1epoch loss: 0.1403175803218403
1epoch loss: 0.13982271013818287
1epoch loss: 0.13844243153770702
1epoch loss: 0.1372594161404947
1epoch loss: 0.13693721700956035
1epoch loss: 0.13726230938535916
1epoch loss: 0.13675700249016076
1epoch loss: 0.13692868274312436
1epoch loss: 0.13826964409903858
1epoch loss: 0.13765156910536266
1epoch loss: 0.13666977752814025
1epoch loss: 0.13681827195988003
1epoch loss: 0.13593342950848566


Epoch:  10%|█         | 2/20 [1:49:04<15:41:54, 3139.72s/it]

...

Epoch:  90%|█████████ | 18/20 [15:23:38<1:38:26, 2953.22s/it]

18epoch loss: 6.109276910137851e-06
18epoch loss: 0.011822838373672815
18epoch loss: 0.008052835978643468
18epoch loss: 0.009698969755850993
18epoch loss: 0.009932743235720014
18epoch loss: 0.010677051638307906
18epoch loss: 0.010179665626137629
18epoch loss: 0.010082219929933854
18epoch loss: 0.009838267705593727
18epoch loss: 0.009296356699436785
18epoch loss: 0.009589929846220663
18epoch loss: 0.009551733702201602
18epoch loss: 0.008983928652083294
18epoch loss: 0.00914125485405062
18epoch loss: 0.009034382514421005
18epoch loss: 0.00890624743524677
18epoch loss: 0.008948986403667022
18epoch loss: 0.008903065558291045
18epoch loss: 0.00869791022497675
18epoch loss: 0.008585539964981152
18epoch loss: 0.008557495076888869
18epoch loss: 0.008527439435911211
18epoch loss: 0.008690235737563733
18epoch loss: 0.008632488081525991
18epoch loss: 0.008411602816089563
18epoch loss: 0.008624376329850702
18epoch loss: 0.008701413930790039


Epoch:  95%|█████████▌| 19/20 [16:12:55<49:14, 2954.32s/it]  

19epoch loss: 0.00018654382438398898
19epoch loss: 0.011227520171333633
19epoch loss: 0.011887181312152792
19epoch loss: 0.012592037845984382
19epoch loss: 0.011079731446928594
19epoch loss: 0.010283380245244977
19epoch loss: 0.010067659976776316
19epoch loss: 0.009476075752871002
19epoch loss: 0.008828560373742073
19epoch loss: 0.008281208019345924
19epoch loss: 0.008165714428255729
19epoch loss: 0.009063670515746504
19epoch loss: 0.00929253076349785
19epoch loss: 0.009231522420546428
19epoch loss: 0.009170287471323571
19epoch loss: 0.008929619658945735
19epoch loss: 0.009352687166659409
19epoch loss: 0.009027382925977562
19epoch loss: 0.008893892449996
19epoch loss: 0.008763936685610964
19epoch loss: 0.008917171130896975
19epoch loss: 0.008786972284632195
19epoch loss: 0.00873491856303588
19epoch loss: 0.00873052417237153
19epoch loss: 0.008568023868725193
19epoch loss: 0.00856221605094252
19epoch loss: 0.008484285103997225


Epoch: 100%|██████████| 20/20 [17:02:24<00:00, 3067.21s/it]

In [11]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [12]:
# p_encoder = retriever.p_encoder
# q_encoder = retriever.q_encoder
with torch.no_grad() :
    p_encoder.eval()

    p_embs = []
    for p in tqdm(corpus) :
        p = tokenizer(p, padding='max_length', truncation=True, return_tensors='pt').to('cuda')
        p_emb = p_encoder(**p).to('cpu').numpy()
        p_embs.append(p_emb)
p_embs = torch.Tensor(p_embs).squeeze()

In [15]:
import pickle

file_path = '/opt/ml/custom/passage_embedding_special_customsample_augmentation_20.bin'
with open(file_path, 'wb') as file :
    pickle.dump(p_embs, file)

In [13]:
p_encoder.cpu()
del p_encoder
torch.cuda.empty_cache()

In [16]:
torch.save(q_encoder, '/opt/ml/custom/q_encoder_special_customsample_augmentation_20.pt')

## Get Relavant Documnet

In [6]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [7]:
model_checkpoint = "klue/bert-base"

# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
import pickle
with open('/opt/ml/custom/passage_embedding_special_shuffle_100.bin', 'rb') as file :
    p_embs = pickle.load(file)
p_embs = p_embs

q_encoder = torch.load('/opt/ml/custom/q_encoder_special_customsample_70.pt')

In [18]:
def get_relavant_doc(queries, q_encoder, p_embs, k=1) :
    with torch.no_grad() :
        q_encoder.eval()
        q_seqs_val = tokenizer(queries, padding='max_length',truncation=True,return_tensors='pt').to(device)
        q_emb = q_encoder(**q_seqs_val).to('cpu')
    dot_prod_scores = torch.mm(q_emb, p_embs.T)
    sort_result = torch.sort(dot_prod_scores, dim=1, descending=True)

    scores, ranks = sort_result[0], sort_result[1]

    result_scores = []
    result_indices = []
    for i in range(len(ranks)) :
        result_scores.append(scores[i].tolist()[:k])
        result_indices.append(ranks[i].tolist()[:k])
    
    return result_scores, result_indices

### 연산 시작 - Augmentation/Batch_size/epoch:20/CustomSampling/512

In [20]:
original_dataset = load_from_disk('/opt/ml/data/train_dataset')

In [37]:
doc_scores, doc_indices = get_relavant_doc(original_dataset['validation']['question'], q_encoder, p_embs, k = 500)

total = []
for idx, example in enumerate(
        tqdm(original_dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": doc_indices[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in doc_indices[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

cqas_100 = pd.DataFrame(total)

In [38]:
correct_length_100 = []
for i in range(len(cqas_100)) :
    if cqas_100['original_context'][i] in cqas_100['context'][i] :
        correct_length_100.append(i)
print(len(correct_length_100) / len(original_dataset['validation']))

0.9541666666666667


In [39]:
cqas_100.to_csv('/opt/ml/custom/valid_dpr_b16_speical_customsampling_augmentation_e20_t500.csv', index = False)

### 연산 시작 - Batch_size 16/epoch:10/CustomSampling


In [23]:
doc_scores, doc_indices = get_relavant_doc(dataset['validation']['question'], q_encoder, p_embs, k = 50)

total = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": doc_indices[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in doc_indices[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

cqas_100 = pd.DataFrame(total)

In [24]:
correct_length_100 = []
for i in range(len(cqas_100)) :
    if cqas_100['original_context'][i] in cqas_100['context'][i] :
        correct_length_100.append(i)
print(len(correct_length_100) / len(dataset['validation']))

0.6916666666666667


In [25]:
cqas_100.to_csv('/opt/ml/custom/valid_dpr_b_16_speical_customsampling_e10_t50.csv', index = False)

### 연산 시작 - Batch_size 16/epoch:10/CustomSampling/384


In [21]:
doc_scores, doc_indices = get_relavant_doc(dataset['validation']['question'], q_encoder, p_embs, k = 50)

total = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": doc_indices[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in doc_indices[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

cqas_100 = pd.DataFrame(total)

In [22]:
correct_length_100 = []
for i in range(len(cqas_100)) :
    if cqas_100['original_context'][i] in cqas_100['context'][i] :
        correct_length_100.append(i)
print(len(correct_length_100) / len(dataset['validation']))

0.7166666666666667


In [23]:
cqas_100.to_csv('/opt/ml/custom/valid_dpr_b_16_speical_customsampling_e70_t50.csv', index = False)